In [1]:
# Import required libraries
import io
import numpy as np
import torch
import trimesh
from ultralytics import YOLO
import cv2
from PIL import Image
from depth_pro import create_model_and_transforms, load_rgb
from pycocotools import mask as maskUtils
import open3d as o3d
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
import time

In [2]:
# Camera intrinsic parameters
fx = 3098.0475
fy = 3097.4233
cx = 2031.6249
cy = 1524.7746

# Camera extrinsic parameters
R = np.array([
    [0.965286851297566, 0.248307782214057, -0.0810218489295801],
    [-0.157358887774226, 0.800448714153273, 0.578377071164504],
    [0.208469362618323, -0.545550273854123, 0.811736055345087]
])
T = np.array([-101.197833337719, -31.2983198623872, 356.795325964250]).reshape(3, 1)

# Initialize segmentation and depth models

seg_model = YOLO('yolo11x-seg.pt')
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
depth_model, transform = create_model_and_transforms(device=device, precision=torch.half)

In [6]:
def calibrate_camera(image_paths):
    """Calibrate camera using checker board images"""
    # Checker board dimensions
    CHECKERBOARD = (7, 10)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare object points
    objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1,2)

    objpoints = []  # 3D points in real world space
    imgpoints = []  # 2D points in image plane

    for image_path in image_paths[:-1]:  # Process all images except the last one
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
        
        if ret:
            corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
            objpoints.append(objp)
            imgpoints.append(corners2)

    # Calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    # Get extrinsic parameters from last image
    last_img = cv2.imread(image_paths[-1])
    last_gray = cv2.cvtColor(last_img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(last_gray, CHECKERBOARD, None)
    
    if ret:
        corners2 = cv2.cornerSubPix(last_gray, corners, (11,11), (-1,-1), criteria)
        _, rvec, tvec = cv2.solvePnP(objp, corners2, mtx, dist)
        R, _ = cv2.Rodrigues(rvec)
        T = tvec

    return mtx, dist, R, T

def get_extrinsic_params(image_path):
    """Get extrinsic parameters from an image using camera intrinsic parameters"""
    # Checker board dimensions (7x10)
    CHECKERBOARD = (7, 10)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # Prepare object points
    objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1,2)
    # Create camera matrix from intrinsic parameters
    camera_matrix = np.array([[fx, 0, cx],
                             [0, fy, cy],
                             [0, 0, 1]], dtype=np.float32)
    
    # Process image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
    
    if ret:
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        # Assuming no distortion
        dist_coeffs = np.zeros((4,1))
        # Get rotation and translation vectors
        _, rvec, tvec = cv2.solvePnP(objp, corners2, camera_matrix, dist_coeffs)
        R, _ = cv2.Rodrigues(rvec)
        T = tvec
        return R, T
    
    return None, None

def generate_cylinder_obj(radius: float, height: float):
    """Generate cylinder mesh object"""
    # Create cylinder using trimesh
    cylinder = trimesh.creation.cylinder(radius=radius, height=height, sections=32)
    # Generate memory stream
    obj_stream = io.BytesIO()
    cylinder.export(obj_stream, file_type='obj')
    # Convert BytesIO to string
    obj_stream.seek(0)
    return obj_stream.read().decode('utf-8')

def generate_mask_COCO(image_path: str):
    """Generate COCO format segmentation mask"""
    results = seg_model.predict(source=image_path, show=False)
    result = results[0]
    if result.masks is not None and result.boxes is not None:
        mask = result.masks.xy[0]
        segmentation = [float(coord) for segment in mask.tolist() for coord in segment]
    return segmentation

def generate_depthmap(image_path: str):
    """Generate depth map from RGB image"""
    try:
        image, _, f_px = load_rgb(image_path)
    except Exception as e:
        return np.array([])
    prediction = depth_model.infer(transform(image), f_px=f_px)
    depth = prediction["depth"].detach().cpu().numpy().squeeze()
    return depth

def create_point_cloud(depth_map, mask):
    """Create point cloud from depth map and mask"""
    # Get valid points within 90th percentile depth threshold
    depth_threshold = np.percentile(depth_map[depth_map * mask > 0], 95)
    valid_indices = np.where((depth_map * mask > 0) & (depth_map <= depth_threshold))
    depths = depth_map[valid_indices]
    xs, ys = np.meshgrid(np.arange(depth_map.shape[1]), np.arange(depth_map.shape[0]))

    # Convert to camera coordinates
    points_camera = np.vstack((
        (xs[valid_indices] - cx) * depths / fx,
        (ys[valid_indices] - cy) * depths / fy,
        depths
    ))
    
    # Transform to world coordinates
    return R.T @ (points_camera - T)

def cluster_points(points):
    """Cluster points using DBSCAN"""
    # Project to 2D space
    xy_dist = np.sqrt(points[0]**2 + points[1]**2)
    points_2d = np.vstack((xy_dist, points[2])).T
    
    # Apply DBSCAN clustering
    points_2d_scaled = StandardScaler().fit_transform(points_2d)
    labels = DBSCAN(eps=0.2, min_samples=40).fit_predict(points_2d_scaled)
    
    return labels != -1  # Return valid points mask

def generate_3d_AABB(segmentation, depth, visualization=False):
    """Generate 3D bounding box from segmentation and depth"""
    # Create binary mask
    mask = np.zeros_like(depth, dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(segmentation).reshape(-1, 2).astype(int)], 1)

    # Create point cloud
    points_world = create_point_cloud(depth, mask)

    # Apply downsampling
    sample_size = min(10000, points_world.shape[1])
    sample_indices = np.random.choice(points_world.shape[1], size=sample_size, replace=False)
    sampled_points = points_world[:, sample_indices]

    # Remove noise using clustering
    valid_mask = cluster_points(sampled_points)
    valid_points = sampled_points[:, valid_mask].T

    # Calculate bounding box
    valid_cloud = o3d.geometry.PointCloud()
    valid_cloud.points = o3d.utility.Vector3dVector(valid_points)
    aabb = valid_cloud.get_axis_aligned_bounding_box()
    
    # Calculate dimensions
    size = aabb.get_max_bound() - aabb.get_min_bound()
    # Visualization (optional)
    if visualization:  # Toggle visualization
        aabb.color = (0, 1, 0)  # Green bounding box
        valid_cloud.paint_uniform_color([1, 0, 0])  # Red points
        o3d.visualization.draw_geometries(
            [valid_cloud, aabb],
            window_name="Clustering Result without Noise",
            width=800, height=600
        )

    return size

def generate_3d_OBB(segmentation, depth, visualization=False):
    """Generate 3D bounding box from segmentation and depth"""
    # Create binary mask
    mask = np.zeros_like(depth, dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(segmentation).reshape(-1, 2).astype(int)], 1)

    # Create point cloud
    points_world = create_point_cloud(depth, mask)

    # Apply downsampling
    sample_size = min(10000, points_world.shape[1])
    sample_indices = np.random.choice(points_world.shape[1], size=sample_size, replace=False)
    sampled_points = points_world[:, sample_indices]

    # Remove noise using clustering
    valid_mask = cluster_points(sampled_points)
    valid_points = sampled_points[:, valid_mask].T

    # Calculate bounding box
    valid_cloud = o3d.geometry.PointCloud()
    valid_cloud.points = o3d.utility.Vector3dVector(valid_points)
    obb = valid_cloud.get_oriented_bounding_box()
    
    # Calculate dimensions
    size = obb.get_max_bound() - obb.get_min_bound()
    # Visualization (optional)
    if visualization:  # Toggle visualization
        obb.color = (0, 1, 0)  # Green bounding box
        valid_cloud.paint_uniform_color([1, 0, 0])  # Red points
        o3d.visualization.draw_geometries(
            [valid_cloud, obb],
            window_name="Clustering Result without Noise",
            width=800, height=600
        )

    return size

In [ ]:
R, T = get_extrinsic_params("assets/example6.jpg")
rgb_path = "assets/example6.jpg"
segmentation = generate_mask_COCO(rgb_path)
start_time = time.time()
depth = generate_depthmap(rgb_path)
print("Depth map generation time: ", time.time() - start_time)

size = generate_3d_AABB(segmentation, depth, True)

print(size)


image 1/1 /Users/yoonseok/Documents/Coding/Gencycle/python/assets/example6.jpg: 480x640 1 book, 1771.6ms
Speed: 17.5ms preprocess, 1771.6ms inference, 21.0ms postprocess per image at shape (1, 3, 480, 640)
Depth map generation time:  31.166950941085815
